# Исследование жилой недвижимости г. Саратова

## Очистка и подготовка данных

### План действий:

1. Получить полный список столбцов и их характеристики (тип и количество значений) и удалить лишние
2. Удалить строки, в которых значение столбца 'Тип' пустое (=='')
3. Объединить столбцы, которые несут одинаковую информацию если они есть
4. Преобразовать числовые значения из строк в соответствующий числовой тип
5. Разделить датасет на два - квартиры/комнаты и дома/части домов
6. Удалить лишний текст в столбцах где должны быть только числовые значения (с помощью регулярных выражений)
7. Разделить столбец этаж/этажность на два соответствующих
8. Проверить датасеты на наличие повторений (по url)

In [2]:
import pandas as pd
import seaborn as sns

### Загружаем датасет и смотрим его основные характеристики

In [93]:
raw_data = pd.read_json('living_realty_raw.json', orient='columns')

# код для сохранения в csv
# raw_data.to_csv('cleaned_data.csv', sep=';', index = None, header=True)

In [94]:
raw_data.head()

,url,Адрес,Балкон,Вторичное жилье,Высота потолков,Год постройки,Городской телефон,Дата сдачи,Дом(cтроение),Доп. информация,...,Санузел,Собственность,Создано,Стадия строительства,Тип,Торг,Цена,Цена за м²,Этаж/этажей в доме,Этажей
0,https://kvadrat64.ru/sellflat-499534.html,"в ЖК «Крылья», ул. Чернышевского, 137а, ОКТЯБР...",лоджия,NaN,"потолок 2,8 м",NaN,NaN,2017 год постройки,дом монолитно-каркасный,NaN,...,2 санузла,NaN,04-07-2018,дом сдан,3-х комнатная квартира,NaN,5 000 000,56 818,этаж 6/18,NaN
1,https://kvadrat64.ru/sellflat-499537.html,"в ЖК «Крылья», ул. Чернышевского, 137а, ОКТЯБР...",лоджия,NaN,"потолок 2,8 м",NaN,NaN,2017 год постройки,дом монолитно-каркасный,NaN,...,2 санузла,NaN,04-07-2018,дом сдан,3-х комнатная квартира,NaN,4 250 000,48 295,этаж 6/18,NaN
2,https://kvadrat64.ru/sellflat-495453.html,"в ЖК «Крылья», ул. Им Н. Г. Чернышевского, 137...",2 лоджии,NaN,"потолок 2,8 м",NaN,NaN,2017 год постройки,дом каркасно-монолитный,NaN,...,2 санузла,NaN,07-06-2018,дом сдан,4-х комнатная квартира,NaN,4 860 000,43 393,этаж 4/18,NaN
3,https://kvadrat64.ru/sellflat-548321.html,"ул. Железнодорожная, 17, ФРУНЗЕНСКИЙ, Саратов",есть балкон,2001 год постройки,NaN,NaN,NaN,NaN,дом кирпичный,NaN,...,NaN,NaN,30-06-2019,NaN,1 комнатная квартира,разумный торг,2 100 000,46 667,этаж 6/7,NaN
4,https://kvadrat64.ru/sellflat-543709.html,"ул. Большая Садовая,, 160/80, КИРОВСКИЙ, Сарат...",лоджия,NaN,NaN,NaN,NaN,NaN,дом кирпичный,NaN,...,санузел совмещенный,NaN,15-05-2019,NaN,1 комнатная квартира,разумный торг,1 550 000,46 970,этаж 2/9,NaN


In [95]:
raw_data.shape

(12607, 36)

In [96]:
raw_data.columns

Index(['url', 'Адрес', 'Балкон', 'Вторичное жилье', 'Высота потолков',
       'Год постройки', 'Городской телефон', 'Дата сдачи', 'Дом(cтроение)',
       'Доп. информация', 'Жилая', 'Застройщик', 'Или меняю на', 'Ипотека',
       'Коммуникации', 'Комнаты', 'Кухня', 'Лифт', 'Мусоропровод', 'Обновлено',
       'Окна', 'Планировка квартиры', 'Площадь', 'Площадь общая',
       'Площадь участка', 'Продажа', 'Санузел', 'Собственность', 'Создано',
       'Стадия строительства', 'Тип', 'Торг', 'Цена', 'Цена за м²',
       'Этаж/этажей в доме', 'Этажей'],
      dtype='object')

### Удаляем лишние для всех типов объектов столбцы

In [97]:
columns_to_remove = ['Городской телефон', 'Доп. информация', 'Или меняю на', 'Собственность', 'Площадь общая', 'Цена за м²']
raw_data = raw_data.drop(columns=columns_to_remove)
raw_data.shape

(12607, 30)

### Получаем все типы объектов и разделяем их

In [99]:
raw_data['Тип'].unique()

array(['3-х комнатная квартира', '4-х комнатная квартира',
       '1 комнатная квартира', 'часть дома', '2-х комнатная квартира',
       'комната в многокомн. квартире', 'коттедж', 'дом',
       '5 комнатная квартира', 'комната в 4-х комн. квартире',
       'комната в 3-х комн. квартире', 'таунхаус',
       'комната в коммун. квартире', '2 комнаты в 4-х комн. квартире',
       'комната в общежитии', '2 комнаты в 3-х комн. квартире', '',
       'малосемейка', '6 комнатная квартира',
       '2 комнаты в коммун. квартире', '1.5 комнатная квартира',
       'комната в 2-х комн. квартире', '3 комнаты в общежитии',
       'гостинка/секционка', '1 комнатная квартира 34.',
       '12 комнатная квартира', 'комната в 1.5 комн. квартире',
       '2 комнаты в многокомн. квартире', '2 комнаты в частном доме',
       '9 комнатная квартира', '3 комнаты в коммун. квартире',
       '2 комнаты в общежитии', 'комната в 4-х комн. квартире 14',
       '1 комнатная квартира 26.4', '9 комнат в общежитии',
   

In [100]:
raw_data[raw_data['Тип']=='малосемейка'].shape

(35, 30)

In [124]:
raw_data.head()

,url,Адрес,Балкон,Вторичное жилье,Высота потолков,Год постройки,Дата сдачи,Дом(cтроение),Жилая,Застройщик,...,Площадь участка,Продажа,Санузел,Создано,Стадия строительства,Тип,Торг,Цена,Этаж/этажей в доме,Этажей
0,https://kvadrat64.ru/sellflat-499534.html,"в ЖК «Крылья», ул. Чернышевского, 137а, ОКТЯБР...",лоджия,NaN,"потолок 2,8 м",NaN,2017 год постройки,дом монолитно-каркасный,44.29,Союз-С,...,NaN,чистая продажа,2 санузла,04-07-2018,дом сдан,3-х комнатная квартира,NaN,5 000 000,этаж 6/18,NaN
1,https://kvadrat64.ru/sellflat-499537.html,"в ЖК «Крылья», ул. Чернышевского, 137а, ОКТЯБР...",лоджия,NaN,"потолок 2,8 м",NaN,2017 год постройки,дом монолитно-каркасный,44.29,Союз-С,...,NaN,чистая продажа,2 санузла,04-07-2018,дом сдан,3-х комнатная квартира,NaN,4 250 000,этаж 6/18,NaN
2,https://kvadrat64.ru/sellflat-495453.html,"в ЖК «Крылья», ул. Им Н. Г. Чернышевского, 137...",2 лоджии,NaN,"потолок 2,8 м",NaN,2017 год постройки,дом каркасно-монолитный,57.6,Союз-С,...,NaN,чистая продажа,2 санузла,07-06-2018,дом сдан,4-х комнатная квартира,NaN,4 860 000,этаж 4/18,NaN
3,https://kvadrat64.ru/sellflat-548321.html,"ул. Железнодорожная, 17, ФРУНЗЕНСКИЙ, Саратов",есть балкон,2001 год постройки,NaN,NaN,NaN,дом кирпичный,23,NaN,...,NaN,чистая продажа,NaN,30-06-2019,NaN,1 комнатная квартира,разумный торг,2 100 000,этаж 6/7,NaN
4,https://kvadrat64.ru/sellflat-543709.html,"ул. Большая Садовая,, 160/80, КИРОВСКИЙ, Сарат...",лоджия,NaN,NaN,NaN,NaN,дом кирпичный,18,NaN,...,NaN,NaN,санузел совмещенный,15-05-2019,NaN,1 комнатная квартира,разумный торг,1 550 000,этаж 2/9,NaN


In [101]:
# содержит текст "квартира" либо "малосемейка" либо "гостинка"
flats_columns = ['1 комнатная квартира', '1.5 комнатная квартира', '2-х комнатная квартира', '3-х комнатная квартира', 
                 '4-х комнатная квартира', '5 комнатная квартира', '6 комнатная квартира', '9 комнатная квартира',
                 '12 комнатная квартира', 
                 'малосемейка', 'гостинка/секционка']

rooms_columns = ['комната в многокомн. квартире', 'комната в 4-х комн. квартире', 'комната в 3-х комн. квартире',
                'комната в коммун. квартире', '2 комнаты в 4-х комн. квартире', 'комната в общежитии', 
                 '2 комнаты в 3-х комн. квартире', '2 комнаты в коммун. квартире', 'комната в 2-х комн. квартире', 
                 '3 комнаты в общежитии', 'комната в 1.5 комн. квартире', '2 комнаты в многокомн. квартире',
                '3 комнаты в коммун. квартире', '2 комнаты в общежитии', '9 комнат в общежитии']

house_columns = ['часть дома', 'коттедж', 'дом', 'таунхаус']

In [125]:
# Удалить объекты с типом '', '2 комнаты в частном доме'
data = raw_data[(raw_data['Тип'] !='')]
data = data[~data['Тип'].str.contains('доме', na=False)]
data.shape

(12599, 30)

In [126]:
data.head()

,url,Адрес,Балкон,Вторичное жилье,Высота потолков,Год постройки,Дата сдачи,Дом(cтроение),Жилая,Застройщик,...,Площадь участка,Продажа,Санузел,Создано,Стадия строительства,Тип,Торг,Цена,Этаж/этажей в доме,Этажей
0,https://kvadrat64.ru/sellflat-499534.html,"в ЖК «Крылья», ул. Чернышевского, 137а, ОКТЯБР...",лоджия,NaN,"потолок 2,8 м",NaN,2017 год постройки,дом монолитно-каркасный,44.29,Союз-С,...,NaN,чистая продажа,2 санузла,04-07-2018,дом сдан,3-х комнатная квартира,NaN,5 000 000,этаж 6/18,NaN
1,https://kvadrat64.ru/sellflat-499537.html,"в ЖК «Крылья», ул. Чернышевского, 137а, ОКТЯБР...",лоджия,NaN,"потолок 2,8 м",NaN,2017 год постройки,дом монолитно-каркасный,44.29,Союз-С,...,NaN,чистая продажа,2 санузла,04-07-2018,дом сдан,3-х комнатная квартира,NaN,4 250 000,этаж 6/18,NaN
2,https://kvadrat64.ru/sellflat-495453.html,"в ЖК «Крылья», ул. Им Н. Г. Чернышевского, 137...",2 лоджии,NaN,"потолок 2,8 м",NaN,2017 год постройки,дом каркасно-монолитный,57.6,Союз-С,...,NaN,чистая продажа,2 санузла,07-06-2018,дом сдан,4-х комнатная квартира,NaN,4 860 000,этаж 4/18,NaN
3,https://kvadrat64.ru/sellflat-548321.html,"ул. Железнодорожная, 17, ФРУНЗЕНСКИЙ, Саратов",есть балкон,2001 год постройки,NaN,NaN,NaN,дом кирпичный,23,NaN,...,NaN,чистая продажа,NaN,30-06-2019,NaN,1 комнатная квартира,разумный торг,2 100 000,этаж 6/7,NaN
4,https://kvadrat64.ru/sellflat-543709.html,"ул. Большая Садовая,, 160/80, КИРОВСКИЙ, Сарат...",лоджия,NaN,NaN,NaN,NaN,дом кирпичный,18,NaN,...,NaN,NaN,санузел совмещенный,15-05-2019,NaN,1 комнатная квартира,разумный торг,1 550 000,этаж 2/9,NaN


In [134]:
# датасет с квартирами 
data_flats = pd.concat([data[data['Тип'].str.contains('квартира', na=False)], 
                        data[data['Тип'].str.contains('малосемейка', na=False)],
                       data[data['Тип'].str.contains('гостинка', na=False)]], ignore_index=True)

In [135]:
# датасет с комнатами
#data_rooms = data[data['Тип'].str.contains('комната', na=False)] + data[data['Тип'].str.contains('комнаты', na=False)]
data_rooms = pd.concat([data[data['Тип'].str.contains('комната', na=False)], 
                       data[data['Тип'].str.contains('комнаты', na=False)]], ignore_index=True)

In [139]:
# датасет с домами
#data_houses = data[data['Тип'].str.contains('дом', na=False)] + data[data['Тип'].str.contains('коттедж', na=False)] + data[data['Тип'].str.contains('таунхаус', na=False)]
data_houses = pd.concat([data[data['Тип'].str.contains('дом', na=False)],
                        data[data['Тип'].str.contains('коттедж', na=False)],
                        data[data['Тип'].str.contains('таунхаус', na=False)]], ignore_index=True)

In [141]:
data_flats.shape[0] + data_rooms.shape[0] + data_houses.shape[0]

12598

### Работаем с датасетом квартир

In [142]:
data_flats.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11224 entries, 0 to 11223
Data columns (total 30 columns):
url                     11224 non-null object
Адрес                   11224 non-null object
Балкон                  6974 non-null object
Вторичное жилье         2430 non-null object
Высота потолков         1743 non-null object
Год постройки           18 non-null object
Дата сдачи              2251 non-null object
Дом(cтроение)           9757 non-null object
Жилая                   11224 non-null object
Застройщик              1240 non-null object
Ипотека                 5879 non-null object
Коммуникации            0 non-null object
Комнаты                 4620 non-null object
Кухня                   11223 non-null object
Лифт                    5074 non-null object
Мусоропровод            1416 non-null object
Обновлено               11224 non-null object
Окна                    4465 non-null object
Планировка квартиры     2396 non-null object
Площадь                 11224 non-nu

In [143]:
columns_to_remove_flats = ['Этажей']
data_flats = data_flats.drop(columns=columns_to_remove_flats)
data_flats.shape

(11224, 29)

In [147]:
data_flats.columns

Index(['url', 'Адрес', 'Балкон', 'Вторичное жилье', 'Высота потолков',
       'Год постройки', 'Дата сдачи', 'Дом(cтроение)', 'Жилая', 'Застройщик',
       'Ипотека', 'Коммуникации', 'Комнаты', 'Кухня', 'Лифт', 'Мусоропровод',
       'Обновлено', 'Окна', 'Планировка квартиры', 'Площадь',
       'Площадь участка', 'Продажа', 'Санузел', 'Создано',
       'Стадия строительства', 'Тип', 'Торг', 'Цена', 'Этаж/этажей в доме'],
      dtype='object')

In [148]:
data_flats.to_csv('data_flats.csv', sep=';', index = None, header=True)